# Unsupervised Clustering using Hidden Markov Model 

## Import packages

In [55]:
import mne
import numpy as np
import pandas as pd
import seaborn as sns
from hmmlearn import hmm
from simpl_eeg import (
    connectivity,
    eeg_objects,
    raw_voltage,
    topomap_2d,
    topomap_3d_brain,
    topomap_3d_head,
)
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [56]:
%matplotlib inline

## Introduction

Electroencephalograms (EEG) is an electrophysiological measurement method used to examine the electrical activity of the brain and represent it as location-based channels of waves and frequencies. Essentially, the EEG data from our dataset is recorded from 19 electrodes nodes for 1.5 hours. Therefore, the EEG data is in high dimensionality and could be represented as a multivariate time series data. If we present the data in a tabular format, the number of rows would be the time stamps and the number of columns would be the different electrodes. As we have 1.5 hrs experiment data and each seconds is recorded at 2048 Hz, which means we have 2048 EEG data readings per second, our dataset is large with at least 1 million rows.

## Objectives

EEG data is widely use in diagnosing brain disorders such as epilepsy and brain damage from head injuries, however, with the complexity of data and its dynamic changes over time, it is hard to identify any significant patterns by simply reading the data or visualizing it. The main objective of this strech goal is to find similar patterns from the combination of EEG signals of all 19 electrodes for a given time section from the dataset. In plain English, it is to cluster the brain states for different time periods in the data.

## Why Hidden Markov Model

A Markov model assumes that the future is conditionally independent of the past given the present (Daniel & James, 2020), with the probability shown below:
$$P(S_i |S_1...S_{i−1}) = P(S_i|S_{i−1})$$ where $S_i$ is the state at time i.
</br>
A hidden Markov model (HMM) relates a sequence of observations to a sequence of hidden states that explain the observations (Daniel & James, 2020). For the EEG data, the sequence of observations is the EEG data per time frame and the sequence of hidden states would be the brain states in the dataset. Since the brain activities at time $i$ is less likely to highly correlate to brain activitiees before time $i-1$, the Markov model assumption would be satisfied at this case and therefore we would like to try to apply the hidden Markov model to EEG data.

Since we don't know have labeled data or pre-defined brain states, we would need to use unsupervised HMM for this task. The process of finding the sequence of hidden states given the sequence of observations using HMM is called decoding (Daniel & James, 2020) and the `Viterbi` algorithm is commonly used for decoding. Therefore, for this notebook, I would use the `Viterbi` algorithm in the HMM model for finding the potential brain states. 

A hidden Markov model consists of 5 components:
- the state space: a set of hidden states
- the sequence of observation
- the transition probability matrix: the probability transitioning from state $i$ to state $j$
- the emission probabilities: conditional probabilities for all observations given a hidden state
- the initial probability over states: the probability for the Markov model starts at state $i$

The goal for this task is to explore the set of hidden states (the state space) and the transition probability matrix of the EEG data using hidden Markov model. 

## Read in the Data

In [57]:
raw_full = mne.io.read_raw_eeglab("../../data/927/fixica.set")

Reading C:\Users\Yiki\Documents\UBC\MDS\Homework\capstone\simpl_eeg_capstone\data\927\fixica.fdt


<ipython-input-57-99b5e4cf8c83>:1: RuntimeWarning: Data file name in EEG.data (927 fix ica correct.fdt) is incorrect, the file name must have changed on disk, using the correct file name (fixica.fdt).
  raw_full = mne.io.read_raw_eeglab("../../data/927/fixica.set")


In [58]:
entire_df = raw_full.to_data_frame()

In [59]:
channel_names=raw_full.ch_names

## Exploratory Data Analysis

In [ ]:
entire_df.describe()

In [ ]:
sns.violinplot(data=entire_df['Fp1'].values)

In [ ]:
sns.violinplot(data=entire_df['O2'].values)

According to the EDA, the EEG data is following Gaussian distribution, therefore, a GaussianHMM model would be used. However, there are clearly some outliers in the data. My next step is to remove the outliers of the data based on the expertise suggestions from the partner to keep EEG data which falls into (-50, 50) range.

## Data preprocessing

In [ ]:
# drop rows where all values are zero
cleaned_df = entire_df.loc[(entire_df[channel_names] != 0).all(axis=1)]

# drop the outliers of dataset (only keep rows where EEG voltage is between -50 to 50)
df_no_outliers = cleaned_df.loc[((cleaned_df[channel_names] <=50) & (cleaned_df[channel_names] >= -50)).all(axis=1)]

# chunk the data into per second (each second has 2048 readings (rows))
df_second = np.split(df_no_outliers, range(2048, len(df_no_outliers), 2048))

# for each second, randomly sampled 10 time stamps (the original dataset is too big, wants to sample a smaller dataset for exploration)
df_second_resample={}
for second in range(len(df_second)):
    df_second_resample[second] = df_second[second].sample(10, random_state=2020, axis=0).sort_values(by="time")
df_resampled = pd.concat([values for key, values in df_second_resample.items()])

## Building Models

Based on the suggestion from the partner, we would like to explore the data in per 5 second interval. 

In [ ]:
five_second_df = np.split(df_resampled, range(50, len(df_resampled), 50))

In [ ]:
# since HMM model only takes in (n_sample, n_feature) array, reshape the data into an array where each sample has 50 time stamps (5 seconds data)
chunked_list = []
for i in range(len(five_second_df)):
    chunked_list.append(np.array(five_second_df[i].iloc[:, 1:]).flatten())
chunked_array = np.array(chunked_list)

As mentioned above, the EEG data follows Gaussian distribution and is continous, we would use the `GaussianHMM` model from `hmmlearn` package. Since we don't know the number of brain states from the model, we would like to start with some random numbers.

In [ ]:
# n_components is the number of hidden states (number of brain states)
chunked_model = hmm.GaussianHMM(n_components=4)

In [ ]:
chunked_model.fit(chunked_array)

In [ ]:
chunked_result = chunked_model.decode(chunked_array, algorithm="viterbi")

### Check model output

In [ ]:
# the metric that hmmlearn package itself used to evaluate the model
print(f"The log probability for this 4-cluster model is {chunked_result[0]:0.4f}")

In [ ]:
print(f"The prior probability for this model is: {chunked_model.startprob_}")

In [ ]:
print(f"The transmission probability matrix of this model is: \n {chunked_model.transmat_}")

In [ ]:
# add lables back to the df
df_result = five_second_df.copy()
for i in range(len(df_result)):
    df_result[i]=df_result[i].assign(cluster = chunked_result[1][i])

In [ ]:
df_result=pd.concat([df_result[i] for i in range(len(df_result))])

In [ ]:
df_result

### Visualize output

#### Visualize the average voltage for each cluster

In [ ]:
cluster_0 = df_result[df_result['cluster'] == 0]
cluster_1 = df_result[df_result['cluster'] == 1]
cluster_2 = df_result[df_result['cluster'] == 2]
cluster_3 = df_result[df_result['cluster'] == 3]

In [ ]:
topomap_2d.plot_topomap_2d(raw_full, cluster_0.iloc[:, 1:20].mean().values*1e-6, mark="channel_name",  cmin=-0.5, cmax=0.5)

In [ ]:
topomap_2d.plot_topomap_2d(raw_full, cluster_1.iloc[:, 1:20].mean().values*1e-6, mark="channel_name", cmin=-1, cmax=1)

In [ ]:
topomap_2d.plot_topomap_2d(raw_full, cluster_2.iloc[:, 1:20].mean().values*1e-6, mark="channel_name",  cmin=-0.3, cmax=0.3)

In [ ]:
topomap_2d.plot_topomap_2d(raw_full, cluster_3.iloc[:, 1:20].mean().values*1e-6, mark="channel_name", cmin=-2, cmax=2)

#### Visualize in an animated way

In [ ]:
cluster_array = np.array(df_result.cluster)
change_index = np.where(cluster_array[:-1] != cluster_array[1:])[0]

In [ ]:
first_batch = df_result.iloc[:change_index[0]+1]

In [ ]:
try_batch = first_batch.iloc[:5]

In [ ]:
%%capture
fig, ax = plt.subplots()
def animate(frame_number):
    fig.clear()
#     ax.clear()
    fig = topomap_2d.plot_topomap_2d(raw_full, try_batch.iloc[frame_number, 1:20], mark="channel_name")
#     plot_static=plt.plot(1,2)
    return fig
ani = animation.FuncAnimation(
        fig,
        animate,
        frames=range(len(try_batch)),
        blit=True
    )

In [ ]:
from IPython.core.display import HTML

HTML(ani.to_jshtml())

## Next Steps

Due to the limited time and efforts that we could allocate to this task, there are other potential useful approaches to try for this task but haven't been implemented yet. 

- Data preprocessing: instead of sampling only 10 time stamps per second, increase the sampling rate so that it could capture more dynmaics from each second to provide a more accurate resutl.

- Feature engineering: instead of only using the raw voltage data for model input, include some engineered features that could provide a better representation of the temporal dependencies of the data such as the following:
    - apply rolling mean for each 5 second data chunks rather than simply taking the mean of each 5 second data chunks
    - use the sliding window approach to slide the per 5 second data 


- Literature review: read through more literature articles to define a better metric to evaluate the model

- Hyperparameter tuning: currently, there isn't a better way to find the optimal `# of cluster` in the model other than finish fitting the model and visualizing the output to check. Use the metric that we could locate from the previous objective to tune the hyperparameter.

## Attribution

- Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2020. All
rights reserved. Draft of December 30, 2020.
